# Poisson editing
Adapted by Nicolas Papadakis (IMB) and Charles Dossal (INSA Toulouse) from works by William Emmanuel and Pierre Bénard (LaBRI)

## Introduction
The goal of this assignment is to apply the Poisson editing algorithm [1] for image blending.

In the following, $T$ is a target image,  $S$ a source image,  and a binary mask representing an area $\Omega$ of $S$ to copy in $T$. All images are defined on the same domain $D$ of sizee $M\times N$.

The idea of [1] is to copy the spatial gradients $\nabla S$ of the source image inside $T$, and not the color values $S$. As illustrated below, this gives more realistic blendings.


<table align="center"><tr><td><img src="./img/target_Boat.png" style="width: 200px;"></td><td><img src="./img/source_Kraken.png" style="width: 200px;"></td><td><img src="./img/mask_kraken.png" style="width: 200px;"></td><td><img src="./img/naive.png" style="width: 200px;"></td><td><img src="./img/poisson_blending.png" style="width: 200px;"></td></tr>
<tr><td>Target $T$</td><td>Source $S$</td><td>Mask</td><td>Naive copy/paste of $S$</td><td>Poisson blending [1]: copy of $\nabla S$</td></tr>
</table>





To realize such blending, we find an image $u$ solution of:

$$\min_u \int_\Omega ||\nabla u-\nabla S||^2,$$
under the constraint $u_{D\backslash \Omega}=T$.

This problem can be stated as follow
\begin{equation*}
\min_u \int_\Omega ||\nabla u-\nabla S||^2+\iota_{K}(u)
\end{equation*}
where $K$ is the set of images which coincide with the target out of the mask. 
We can observe that the set $K$ is a closed convex set... Why ?

In this setting we can use the projected gradient which is a particular case of the Forward backward algorithm to solve this problem of fusion of images.

We will study two examples but you can try others on your own.
The data can be found at the following adress :

http://dl.free.fr/rm.pl?h=bOSoNRP4l&i=93426714&s=mwWbk2t533WxFMxD47ERDuR9MlkDPwc8


In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
import matplotlib.pyplot as plt
from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
from PIL import Image



In [ ]:
caselist=['Kraken', 'MonaLisa']

In [ ]:
def chargeData(name):
    if name=='Kraken':
        target=np.array(Image.open("./img/target_Boat.png")).astype(float)
        source=np.array(Image.open("./img/source_Kraken.png")).astype(float)
        mask2=np.array(Image.open("./img/mask_Kraken.png")).astype(float)/255
    if name=='MonaLisa':
        target=np.array(Image.open("./img/Joconde.jpg")).astype(float)
        source=np.array(Image.open("./img/source_Heisenberg.jpeg")).astype(float)
        mask2=np.array(Image.open("./img/mask_joconde.jpeg")).astype(float)/255
    return target,source,mask2

Have a look to the two set of data.

In [ ]:
target,source,mask2=chargeData('MonaLisa')
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask2*255).astype('uint8')).opts(**optionsGray))

In the following we are giving some discrete gradient and associated divergence.

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

Define the two functions Projection and Gradient that will be necessary to compute the projected gradient. What is the Lipschitz constant of the gradient of $x\mapsto \Vert\nabla x -y\Vert_2^2$ ?

In [ ]:
def Proj(im,ma,iref):
    res=im*ma+(1-ma)*iref
    return res
def GradientFonc(x,y):
    g=Gradient(x)
    r1=g[0]-y[0]
    r2=g[1]-y[1]
    res=DivHor(r1)+DivVer(r2)
    return res

In the first step we divide the source and the target into the three chanels

In [ ]:
target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]

Compute then a naive fusion with a simple projection.

In [ ]:
res0=Proj(source0,mask2,target0)
res1=Proj(source1,mask2,target1)
res2=Proj(source2,mask2,target2)
res=target.copy()
res[:,:,0]=res0#np.abs(res0-source0)*mask2
res[:,:,1]=res1#np.abs(res1-source1)*mask2
res[:,:,2]=res2#np.abs(res2-source2)*mask2
res_naive=res.copy()
hv.RGB(res_naive.astype('uint8')).opts(**optionsRGB)

Write a function  FBPoissonEditing that compute the projected gradient on a grayscale image (single color chanel). Don't forget to clip the image at the end.
The function must return the last iterate of the sequence and a curve of the values of iterates (that can be sampled for example with only at most 100 or 200 values) 

In [ ]:
def FBPoissonEditing(targ,sour,ma,step,Niter):
    x=np.copy(sour)
    y=Gradient(sour)
    for k in range(0,Niter):
        temp=x-step*GradientFonc(x,y)
        x=Proj(temp,ma,targ)  
    return np.clip(x,0,255)

In [ ]:
def FBPoissonEditing2(targ,sour,ma,step,Niter):
    x=np.copy(sour)
    y=Gradient(sour)
    diff=max(int(np.floor(Niter/100)),1)
    nbaffichage=int(np.floor(Niter/diff))
    f=np.zeros(nbaffichage)
    count=0 
    for k in range(0,Niter):
        temp=x-step*GradientFonc(x,y)
        x=Proj(temp,ma,targ)
        if np.mod(k,diff)==0 and (count<nbaffichage):
            z=Gradient(x)
            f[count]=np.linalg.norm(z[0]-y[0],'fro')+np.linalg.norm(z[1]-y[1],'fro')
            count=count+1
    return np.clip(x,0,255),f[10:]

Test the function with a step smaller than $1/4$ and 1000 iterations and compare the result with a naive approach. To get the result on a color image, the previous algorithm must be used on each color channel and the 3 output must be gather in a single color image.

In [ ]:
step=1/4
niter=265
res0,f0=FBPoissonEditing2(target0,source0,mask2,step,niter)
res1,f1=FBPoissonEditing2(target1,source1,mask2,step,niter)
res2,f2=FBPoissonEditing2(target2,source2,mask2,step,niter)
res=target.copy()
res[:,:,0]=res0
res[:,:,1]=res1
res[:,:,2]=res2
pn.Row(hv.RGB(res_naive.astype('uint8')).opts(**optionsRGB),hv.RGB(res.astype('uint8')).opts(**optionsRGB),hv.Curve(f0+f1+f2).opts(xaxis=None,toolbar=None))

Using panel, create a dashboard to perform the fusion in real time. The output may be four figures with the source, the target, the fusion and the curve of the decay of the function to minimize. 
The step in the gradient descend used in the algorithm will be $self.step/8$. Hence when the variable step will be set to 1, the step in the gradient descent will be exactly equal to the Lipschitz constant of the gradient of $f$.

In [ ]:
class FBFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(1,bounds=(0.1,4))
    def view(self):
        target,source,mask2=chargeData(self.case)
        step=self.step/8
        niter=self.Niter
        target0=target[:,:,0]
        source0=source[:,:,0]
        target1=target[:,:,1]
        source1=source[:,:,1]
        target2=target[:,:,2]
        source2=source[:,:,2]       
        res0,f0=FBPoissonEditing2(target0,source0,mask2,step,niter)
        res1,f1=FBPoissonEditing2(target1,source1,mask2,step,niter)
        res2,f2=FBPoissonEditing2(target2,source2,mask2,step,niter)
        res=target.copy()
        res[:,:,0]=res0
        res[:,:,1]=res1
        res[:,:,2]=res2
        n=len(f0)/2
        fmin=np.min(f0+f1+f2)
        fmax=np.max(f0+f1+f2)
        strfmin="%2.2f" % fmin
        Courbe=hv.Curve(f0+f1+f2)*hv.Text(n,(fmin+fmax)/2,strfmin)
        return pn.Row(hv.RGB(res.astype('uint8')).opts(**optionsRGB),Courbe.opts(xaxis=None,toolbar=None))

Try the function. 

In [ ]:
fbfusion= FBFusion()
pn.Row(fbfusion.param,fbfusion.view)

Perform a Fusion using FISTA with a parameter $\alpha$ and create the associated dashboard. 


In [ ]:
def FISTAPoissonEditing(targ,sour,ma,step,alpha,Niter):
    x=np.copy(sour)
    y=Gradient(sour)
    diff=max(int(np.floor(Niter/100)),1)
    nbaffichage=int(np.floor(Niter/diff))
    f=np.zeros(nbaffichage)
    count=0 
    xold=x.copy()
    temp=x-step*GradientFonc(x,y)
    x=Proj(temp,ma,targ)
    for k in range(0,Niter):
        x_nes=x+k*(x-xold)/(k+alpha)
        xold=x.copy()
        temp=x_nes-step*GradientFonc(x_nes,y) 
        x=Proj(temp,ma,targ)
        if np.mod(k,diff)==0 and (count<nbaffichage):
            z=Gradient(x)
            f[count]=np.linalg.norm(z[0]-y[0],'fro')+np.linalg.norm(z[1]-y[1],'fro')
            count=count+1
    return np.clip(x,0,255),f[10:]

In [ ]:
target,source,mask2=chargeData('Kraken')
target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]
step=1/8
niter=100
alpha=4
res0,f0=FISTAPoissonEditing(target0,source0,mask2,step,alpha,niter)
res1,f1=FISTAPoissonEditing(target1,source1,mask2,step,alpha,niter)
res2,f2=FISTAPoissonEditing(target2,source2,mask2,step,alpha,niter)
res=target.copy()
res[:,:,0]=res0
res[:,:,1]=res1
res[:,:,2]=res2
fmin=np.min(f0+f1+f2)
fmax=np.max(f0+f1+f2)
strfmin="%2.2f" % fmin
Courbe=hv.Curve(f0+f1+f2)*hv.Text(50,(fmin+fmax)/2,strfmin)
pn.Row(hv.RGB(res.astype('uint8')).opts(**optionsRGB),Courbe.opts(xaxis=None,toolbar=None))

In [ ]:
class FISTAFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(1,bounds=(0.1,4))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData(self.case)
        step=self.step/8
        niter=self.Niter
        target0=target[:,:,0]
        source0=source[:,:,0]
        target1=target[:,:,1]
        source1=source[:,:,1]
        target2=target[:,:,2]
        source2=source[:,:,2]       
        res0,f0=FISTAPoissonEditing(target0,source0,mask2,step,self.alpha,niter)
        res1,f1=FISTAPoissonEditing(target1,source1,mask2,step,self.alpha,niter)
        res2,f2=FISTAPoissonEditing(target2,source2,mask2,step,self.alpha,niter)
        res=target.copy()
        res[:,:,0]=res0
        res[:,:,1]=res1
        res[:,:,2]=res2
        n=len(f0)/2
        fmin=np.min(f0+f1+f2)
        fmax=np.max(f0+f1+f2)
        strfmin="%2.2f" % fmin
        Courbe=hv.Curve(f0+f1+f2)*hv.Text(n,(fmin+fmax)/2,strfmin)
        return pn.Row(hv.RGB(res.astype('uint8')).opts(**optionsRGB),Courbe.opts(xaxis=None,toolbar=None)) 

In [ ]:
fistafusion=FISTAFusion()
pn.Row(fistafusion.param,fistafusion.view)

Compare in a third dashboard the difference between FB and FISTA.
Are the limit on the step the same ?

In [ ]:
class FBvsFISTAFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(1,bounds=(0.1,4))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        target,source,mask2=chargeData(self.case)
        step=self.step/8
        niter=self.Niter
        target0=target[:,:,0]
        source0=source[:,:,0]
        target1=target[:,:,1]
        source1=source[:,:,1]
        target2=target[:,:,2]
        source2=source[:,:,2]       
        res0,f0=FISTAPoissonEditing(target0,source0,mask2,step,self.alpha,niter)
        res1,f1=FISTAPoissonEditing(target1,source1,mask2,step,self.alpha,niter)
        res2,f2=FISTAPoissonEditing(target2,source2,mask2,step,self.alpha,niter)
        res=target.copy()
        res[:,:,0]=res0
        res[:,:,1]=res1
        res[:,:,2]=res2
        n=len(f0)/2
        fmina=np.min(f0+f1+f2)
        fmaxa=np.max(f0+f1+f2)
        strfmina="%2.2f" % fmina
        res0b,f0b=FBPoissonEditing2(target0,source0,mask2,step,niter)
        res1b,f1b=FBPoissonEditing2(target1,source1,mask2,step,niter)
        res2b,f2b=FBPoissonEditing2(target2,source2,mask2,step,niter)
        resb=target.copy()
        resb[:,:,0]=res0b
        resb[:,:,1]=res1b
        resb[:,:,2]=res2b
        fminb=np.min(f0b+f1b+f2b)
        fmaxb=np.max(f0b+f1b+f2b)
        fmin=min(fmina,fminb)
        fmax=max(fmaxa,fmaxb)
        strfminb="%2.2f" % fminb
        courbe1=hv.Curve(f0+f1+f2)
        courbe2=hv.Curve(f0b+f1b+f2b)
        text=hv.Text(n,(0.2*fmin+0.8*fmax),'FB')*hv.Text(n,(0.3*fmin+0.7*fmax),strfminb)\
        *hv.Text(n,(0.7*fmin+0.3*fmax),'FISTA')*hv.Text(n,(0.8*fmin+0.2*fmax),strfmina)
        courbe=courbe2*courbe1*text
        return pn.Column(pn.Row(hv.RGB(res.astype('uint8')).opts(**optionsRGB)\
            ,courbe.opts(xaxis=None,toolbar=None)),hv.RGB(resb.astype('uint8')).opts(**optionsRGB)) 

In [ ]:
fbvsfista=FBvsFISTAFusion()
pn.Row(fbvsfista.param,fbvsfista.view)